### List of all products 

In [1]:
import json
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
import requests

BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
ENDPOINT='/products'


'''
url = BASE_URL + ENDPOINT

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json
  'Accept-Encoding': 'gzip,deflate'
}
productlist = []
listlength = 0
i = 0
while True:
    query = {
        'page[number]': i,
        'page[size]': 200  # maximum
    }
    r = requests.get(url, params=query, headers=headers)
    if r.status_code == 200:
      print('Page ' + str(i) + ' loaded successfully.')

      for product in r.json()['data']:
        productlist.append(product)
        #print('  ' + product['barcode'])

    if listlength == len(productlist):
        break
    print('Product count: ' + str(len(productlist)))
    listlength = len(productlist)
    i += 1
'''

url = BASE_URL + ENDPOINT

query = {
  'page[number]': 1,
  'page[size]': 200
}

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}

r = requests.get(url, params=query, headers=headers)
print("Status: " + str(r.status_code))
if r.status_code == 200:
  print('Page loaded successfully.')
  print('Generated in ' + str(r.json().get('meta', {}).get('generated_in', -1)) + ' milliseconds.')
  print('Next page\'s URL is: ' + r.json()['links'].get('next', 'UNKNOWN'))
  print('Barcodes on this page:')
  for product in r.json()['data']:
    productlist = []
    for x in r.json()['data']:
      productlist.append(x['id'])
    #print('  ' + product['barcode'])
print(productlist)

Status: 200
Page loaded successfully.
Generated in 148 milliseconds.
Next page's URL is: https://www.openfood.ch/api/v3/products?page%5Bnumber%5D=2&page%5Bsize%5D=200
Barcodes on this page:
[1000, 10000, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 1001, 10010, 10011, 10012, 10013, 10014, 10015, 10016, 10017, 10018, 10019, 1002, 10020, 10021, 10022, 10023, 10024, 10025, 10026, 10027, 10028, 10029, 1003, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10037, 10038, 10039, 1004, 10040, 10041, 10042, 10043, 10044, 10045, 10046, 10047, 10048, 10049, 1005, 10050, 10051, 10052, 10053, 10054, 10055, 10056, 10057, 10058, 10059, 10060, 10061, 10062, 10063, 10064, 10065, 10066, 10067, 10068, 10069, 10070, 10071, 10072, 10073, 10074, 10075, 10076, 10077, 10078, 10079, 10080, 10081, 10082, 10083, 10084, 10085, 10086, 10087, 10088, 10089, 10090, 10091, 10092, 10093, 10094, 10095, 10096, 10097, 10098, 10099, 1010, 10100, 10101, 10102, 10103, 10104, 10105, 10106, 10107, 10108, 1010

### Getting list of useful characteristics from id

In [3]:
%%time
BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
info = []
ingredients = []
nutrients = []

for PRODUCT_ID in productlist:
    ENDPOINT='/products/{}'.format(PRODUCT_ID)

    url = BASE_URL + ENDPOINT

    headers = {
      'Authorization': 'Token token=' + KEY,
      'Accept': 'application/json',
      'Content-Type': 'application/vnd.api+json',
      'Accept-Encoding': 'gzip,deflate'
    }

    r = requests.get(url, headers=headers)
    #print ('Status: ' + str(r.status_code))
    item = {}
    item_ing = {}
    item_nut = {}
    data = r.json()['data']
    if r.status_code == 200:
        try:
            item['name'] = item_ing['name'] = item_nut['name'] = data['display_name_translations'].popitem()
            item['id'] = item_ing['id'] = item_nut['id'] = data['id']
            item['barcode'] = item_ing['barcode'] = item_nut['barcode'] = data['barcode']
            
            item['quantity'] = data['quantity']
            item['unit'] = data['unit']
            item['portion'] = data['portion_quantity']
            item['portion unit'] = data['portion_unit']
            item['alcohol'] = data['alcohol_by_volume']
            
            item_ing['ingredients'] = data['ingredients_translations'].popitem()
            
            nuts = {}
            for nut in data['nutrients'].keys():
                nuts[nut] = {'per_day':data['nutrients'][nut]['per_day'],
                             'per_hundred':data['nutrients'][nut]['per_hundred'],
                             'per_portion':data['nutrients'][nut]['per_portion']}
            item_nut['nutrients'] = nuts
        except:
            pass
    info.append(item)
    ingredients.append(item_ing)
    nutrients.append(item_nut)

CPU times: user 5.98 s, sys: 184 ms, total: 6.17 s
Wall time: 44.2 s


In [4]:
pp.pprint(item)
pp.pprint(item_nut)
pp.pprint(item_ing)

{   'alcohol': 0.0,
    'barcode': u'3069538102461',
    'id': 10185,
    'name': (u'fr', u'Dea Harissa Hot Sauce from France'),
    'portion': 0.0,
    'portion unit': u'g',
    'quantity': 120.0,
    'unit': u'g'}
{   'barcode': u'3069538102461',
    'id': 10185,
    'name': (u'fr', u'Dea Harissa Hot Sauce from France'),
    'nutrients': {   u'carbohydrates': {   'per_day': None,
                                           'per_hundred': 5.4,
                                           'per_portion': None},
                     u'energy': {   'per_day': None,
                                    'per_hundred': 324.0,
                                    'per_portion': None},
                     u'energy_kcal': {   'per_day': None,
                                         'per_hundred': 78.0,
                                         'per_portion': None},
                     u'fat': {   'per_day': None,
                                 'per_hundred': 3.2,
                                

In [73]:
ingredients[2]['ingredients'][1]

'asparagi bianchi (Cina), acqua, sale da cucina, antiossidante: acido citrico.'

### Translate ingredients

In [11]:
%%time
from watson_developer_cloud import LanguageTranslatorV2 as LanguageTranslator

language_translator = LanguageTranslator(username='a69f77dc-9248-4069-85b5-1ac1ef6bfd7d',password='UQznP8xcKtuq')

for tmp in ingredients:
    ingredients_ = tmp['ingredients']
    try:
        language_json = language_translator.identify(ingredients_[0])
        language = pd.DataFrame(language_json['languages']).sort_values(by=['confidence'], ascending=False)
        source_lang = language.iloc[0, 1]
        translation = language_translator.translate(text=ingredients_, source=source_lang, target='en')
        ingredients['ingredients'] = json.dumps(translation, indent=2, ensure_ascii=False)
    except:
        print(language_json, ingredients_)

({u'languages': [{u'confidence': 0.0974623, u'language': u'nb'}, {u'confidence': 0.0915791, u'language': u'da'}, {u'confidence': 0.0744227, u'language': u'nn'}, {u'confidence': 0.0695877, u'language': u'is'}, {u'confidence': 0.0668383, u'language': u'sv'}, {u'confidence': 0.0534915, u'language': u'en'}, {u'confidence': 0.0407553, u'language': u'de'}, {u'confidence': 0.0399168, u'language': u'pt'}, {u'confidence': 0.0295877, u'language': u'es'}, {u'confidence': 0.0225441, u'language': u'sq'}, {u'confidence': 0.0211118, u'language': u'fr'}, {u'confidence': 0.0196368, u'language': u'zh'}, {u'confidence': 0.0184548, u'language': u'ro'}, {u'confidence': 0.0149082, u'language': u'lv'}, {u'confidence': 0.0143643, u'language': u'it'}, {u'confidence': 0.0138195, u'language': u'az'}, {u'confidence': 0.0137934, u'language': u'ku'}, {u'confidence': 0.0132626, u'language': u'hu'}, {u'confidence': 0.0129563, u'language': u'tr'}, {u'confidence': 0.0128836, u'language': u'nl'}, {u'confidence': 0.01246

KeyError: 'ingredients'

In [ ]:
pp.pprint(item)
pp.pprint(item_ing)

In [9]:
for ing in ingredients:
    pp.pprint(ing)

{   'barcode': u'8001772000030',
    'id': 1000,
    'ingredients': u'Superior quality durum wheat Pasta\r\n\r\nIngredients: Durum wheat, water\r\n\r\nAll agricultural ingredients are products of organic farming',
    'name': u'Bio Spaghetti '}
{   'barcode': u'7613269357865', 'id': 10000, 'name': u'M Classic Palmherzen'}
{   'barcode': u'7617027558075',
    'id': 10001,
    'name': u'M Classic Weisse Spargelspitzen'}
{   'barcode': u'7613269324638',
    'id': 10002,
    'name': u'Migros S\xe9lection Spargel Stangen'}
{   'barcode': u'7613269357889',
    'id': 10003,
    'name': u'M Classic Artischockenb\xf6den'}
{   'barcode': u'7613312081181',
    'id': 10004,
    'name': u'M Classic Apfel-Bananenmus'}
{   'barcode': u'4104420033740',
    'id': 10005,
    'name': u'Alnatura Apfelmark mit Aprikose'}
{   'barcode': u'4104420034587', 'id': 10006, 'name': u'ALNATURA Apfelmark'}
{   'barcode': u'7617027863834',
    'id': 10007,
    'name': u'M Classic Apfel-Erdbeermus ohne Zuckerzusatz'}
